## Programmatically Deleting Content in ArcGIS Online

Whether you need to remove an old member’s content or delete deprecated items across your organization, manually deleting items is tedious, especially with delete protection and dependencies.

Sounds like a great workflow to automate using the [ArcGIS API for Python](https://developers.arcgis.com/python/), right? But how do we programmatically handle delete protection and dependent items?

The most succinct way to delete an old member’s content is to loop through their folders and call the ArcGIS API for Python’s ContentManager’s [.delete_items()](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager.delete_items) method on all the content in each folder.

Alright, not too bad, but what happens when items have delete protection or dependents?

**False** (that’s all we get from the method).

Everything that can be deleted is deleted, but items with delete protection enabled or with dependents are not.

Okay, but the ContentManager class has another method – .can_delete() – that we could use to handle this, right?

Yes, but now instead of deleting all the content in a folder with one succinct line of code, we’re looping through every item in each folder, checking if it can be deleted, and then if it can’t, dealing with the reason it can’t be deleted. In the case of dependent items, that means we also need another loop to check and delete the dependent items.

Our code is no longer succinct and is becoming harder to read with all the nested for loops and if-else statements. Time to bring in recursion. Recursion allows us to call a function within its own definition. When applicable, this can significantly reduce how much code we have to write to accomplish our goal.

### Import Modules

In [ ]:
from arcgis.gis import GIS, User

### Define recursive function for deleting items with dependents

In [ ]:
def recursive_delete(items: list) -> None:
    """Deletes all items and their dependents."""
    for item in items:
        try:
            dry_run = item.delete(dry_run=True)
        
            if dry_run["can_delete"]:
                item.delete()
            elif dry_run["details"]["message"] == f"Unable to delete item {item.id}. Delete protection is turned on.":
                item.protect(enable=False)
                item.delete()
            elif dry_run["details"]["message"] == "Unable to delete item. This service item has a related Service item":
                recursive_delete(dry_run["details"]["offending_items"])
        except TypeError:
            print(f"Item ({item.id}) no longer exists or is inaccessible.")
    return

### Instantiate GIS

In [ ]:
gis = GIS("home")

### Delete member's content

In [ ]:
# instantiate User object with the old user's username
old_member = User(gis, "<old_username>")

# delete all items in root folder
recursive_delete(old_member.items(max=1000))

# delete items in all other folders
for f in old_member.folders:
    recursive_delete(old_member.items(folder=f, max=1000))